In [138]:
import random
import pandas as pd
import numpy as np
from  flask_restful import Api, Resource, reqparse, abort
from collections import defaultdict
import time
app = Flask(__name__)
api = Api(app)

parser = reqparse.RequestParser()
parser.add_argument('word')
parser.add_argument('wordlist', action='append')


In [75]:
#import CMU dictionary idea for method from:
# https://github.com/aparrish/gen-text-workshop/blob/master/cmu_pronouncing_dictionary_notes.md
rhymedict = defaultdict(list)
worddict = {}
with open('cmudict-0.7b.txt') as f:
    for line in f:
        #skip comments
        if(line[:3]==';;;'):
            continue
        #split elements on the double space
        word, sounds = line[:-1].split('  ')
        sounds = sounds.split(' ')
        if(len(sounds)==1):
            rhyme = sounds[0]
        else:
            rhyme = sounds[-2] + sounds[-1]
        rhymedict[rhyme].append(word)
        worddict[word] = sounds

In [ ]:
class pick_random(Resource):
    '''Returns a random word from a list of inputs.
    
    Does not check for incorrect user inputs.'''
    
    def post(self):
        args = parser.parse_args()
        wordlist = args['wordlist']
        #we will check for non string inputs. This method likely causes problems with a large user base.
        for word in wordlist:
            if(type(word)!=str):
                abort(404, message="Non-string input is not accepted")
                
        return wordlist[random.randrange(len(wordlist))], 201

In [ ]:
class rhymes(Resource):
    '''returns a list of words that rhyme with the input
    
    we'll use a simple definition of rhyming words as words that share the last phonetic sounds'''
    
    def post(self):
        args = parser.parse_args()
        word = args['word']
        rhyme = worddict[word]
        if(len(rhyme)==1):
            rhymelist = rhymedict[rhyme]
        else:
            rhymelist = rhymedict[rhyme[-2]+rhyme[-1]]
        return rhymelist, 201

In [ ]:
#finish building api
api.add_resource(pick_random, '/pickrnd')
api.add_resource(rhymes, '/rhyme')

if __name__ == '__main__':
    app.run()


In [123]:
def pick_rnd(wordlist):
        #we will check for non string inputs. This method likely causes problems with a large user base.
        if(len(wordlist)<2):
            return 0
        for word in wordlist:
            if(type(word)!=str):
                return 0
                #raise ValueError("Non-string input is not accepted")
                
        return wordlist[random.randrange(len(wordlist))]

In [192]:
def rhymes(word):
    #prevent non string inputs
    if(type(word)!= str):
        return 0
    #check for english word
    if(word not in worddict):
        return 0
    sounds = worddict[word.upper()]
    
    if(len(sounds)==1):
        rhyme = sounds[0]
    else:
        rhyme = sounds[-2] + sounds[-1]
            
    if(len(rhyme)==1):
        rhymelist = rhymedict[rhyme]
    else:
        rhymelist = rhymedict[rhyme[-2]+rhyme[-1]]
    return rhymelist

In [193]:
pick_rnd(['awef','aeftet','ujseht'])

'aeftet'

In [194]:
rhymes('hi')

0

In [195]:
def test_pick_rnd():
    #start by testing that the output is in the list of inputs
    wordlist =['start']
    for i in range(20):
        word=''
        for j in range(4):
            word += alphabet[random.randrange(26)]
        wordlist.append(word)
        if(pick_rnd(word) in wordlist):
            print("ERROR: pick_rnd returning value not in input list ")
    print('End Behavior Test')
    
    #check behavior for inputs less than 2
    if(pick_rnd([]) == 0):
        print('Correct Behavior for empty list')
    else:
        print('Fails empty list test')
    if(pick_rnd(['one']) == 0):
        print('Correct Behavior for list with one element')
    else:
        print('Fails one element list test')
        
    #test for integer inputs
    if(pick_rnd(['number', 143]) == 0):
        print('Numerical input test passed')
    else:
        print('Fails numerical element test')
        
    #stress test for repeated calls
    try:
        times = np.array([])
        for i in range(10000):
            t1 = time.time()
            pick_rnd(wordlist)
            t2 = time.time()
            times = np.append(times, (t2-t1))
        print('Average Time (s): {}'.format(np.mean(times)))
        print('Total Time (s): {}'.format(np.sum(times)))
    except Exception as e:
        print(e)
    
    #stress test for large input list
    try:
        for i in range(10000):
            word=''
            for j in range(4):
                word += alphabet[random.randrange(26)]
            wordlist.append(word)
        pick_rnd(wordlist)
    except Exception as e:
        print(e)
        
    print('Testing Completed')      
    

In [199]:
def test_rhymes():
    #testing for input errors
    if(rhymes(413) == 0):
        print('Correct Behavior for numerical input')
    else:
        print('Fails numerical input test')
    if(rhymes('fijef') == 0):
        print('Correct Behavior for gibberish/non english input')
    else:
        print('Fails non-english test')
        
    #stress test
    try:
        times = np.array([])
        for i in range(10000):
            t1 = time.time()
            word = random.choice(list(worddict.keys()))
            rhymes(word)
            t2 = time.time()
            times = np.append(times, (t2-t1))
        print('Average Time (s): {}'.format(np.mean(times)))
        print('Total Time (s): {}'.format(np.sum(times)))
    except Exception as e:
        print(e)
    
    print('Testing Finished')

# Run Tests

In [197]:
test_pick_rnd()

End Behavior Test
Correct Behavior for empty list
Correct Behavior for list with one element
Numerical input test passed
Average Time (s): 3.5030841827392577e-06
Total Time (s): 0.03503084182739258
Testing Completed


In [ ]:
test_rhymes()

Correct Behavior for numerical input
Correct Behavior for gibberish/non english input
